## This demo app shows:
* how to use LlamaIndex, an open source library to help you build custom data augmented LLM applications;
* how to ask Llama questions about breaking news via the You.com live search API and LlamaIndex.
The LangChain package is used to facilitate the call to Llama2 hosted on Replicate.

In [ ]:
!pip install llama-index langchain

In [1]:
# use ServiceContext to configure the LLM used and the custom embeddings 
from llama_index import ServiceContext

# VectorStoreIndex is used to index custom data 
from llama_index import VectorStoreIndex

from langchain.llms import Replicate

In [2]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

 ········


In [3]:
# You.com’s Search API is used to augment LLM responses with the snippets that contains rich info on the query results
# To use the You.com Search API, you can email api@you.com to request an API key. 
YOUCOM_API_KEY = getpass()
os.environ["YOUCOM_API_KEY"] = YOUCOM_API_KEY

 ········


In [4]:
# set llm to be using Llama2 hosted on Replicate
llama2_13b_chat = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"

llm = Replicate(
    model=llama2_13b_chat,
    input={"temperature": 0.01, "max_length": 2000, "top_p": 1},
)

Init param `input` is deprecated, please use `model_kwargs` instead.


In [5]:
# chat about breaking news by first making an API request to the You.com search API
import requests

query = "breaking news"
headers = {"X-API-Key": os.environ["YOUCOM_API_KEY"]}
data = requests.get(
    f"https://api.ydc-index.io/search?query={query}",
    headers=headers,
).json()

In [6]:
# check the query result in JSON
import json

print(json.dumps(data, indent=2))

{
  "hits": [
    {
      "description": "View the latest <strong>news</strong> <strong>and</strong> <strong>breaking</strong> <strong>news</strong> today for U.S., world, weather, entertainment, politics and health at CNN.com.",
      "snippets": [
        "Labor negotiations\n\u2018I feel bad for the nurses, but I feel worse for the patients\u2019: Kaiser members struggle during the strike\nWatch the latest CNN Headlines\nMore top stories\nShirtless man with gun who asked to see governor was arrested at Wisconsin Capitol. He returned with assault rifle after posting bail\nTrump asks judge to dismiss election interference case, citing presidential immunity and Senate acquittal\nAfghan girls can\u2019t attend school past the 6th grade. This secret network of schools is defying the Taliban\nLatest politics\nSenate Democrats reject offer from GOP donor friend of Clarence Thomas for 5 years of financial records\nAd Feedback\nAd Feedback\nCheck these out\nAd Feedback\nCNN Underscored\nAd F

In [7]:
# one way to load the JSON result with key info stored as "snippets"
from llama_index import download_loader

JsonDataReader = download_loader("JsonDataReader")
loader = JsonDataReader()
documents = loader.load_data([hit["snippets"] for hit in data["hits"]])

In [8]:
# another way to add the snippets in the query result to documents - this can be handy if you just need to add a list of text strings to doc
# from llama_index import Document
# snippets = [snippet for hit in data["hits"] for snippet in hit["snippets"]]
# documents = [Document(text=s) for s in snippets]

In [9]:
# use HuggingFace embeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

embeddings = LangchainEmbedding(HuggingFaceEmbeddings())

# create a ServiceContext instance to use Llama2 and custom embeddings
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=800, chunk_overlap=20, embed_model=embeddings)

# create vector store index from the documents created above
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

# create query engine from the index
query_engine = index.as_query_engine(streaming=True)

In [10]:
# ask Llama2 to query about the search result
response = query_engine.query("give me a summary")
response.print_response_stream()

 Sure! Based on the provided context information, here is a summary of the text:

The article discusses several current events and news stories, including:

1. The latest update from Dictionary.com, which highlights new words and phrases influenced by pop culture and social media.
2. A battle over abortion in the US that threatens the President's Emergency Plan for AIDS Relief (PEPFAR).
3. Johnson planning a border trip and migrants being dropped off at Willis Tower.
4. Labor negotiations and a shirtless man with a gun who

In [11]:
# more queries
query_engine.query("what about other breaking news").print_response_stream()

 Based on the provided context information, here are some other breaking news stories:

* North Korea's leader Kim Jong Un has arrived in Russia to meet with President Vladimir Putin amid escalating standoffs with the West. (September 12)
* A series of powerful earthquakes struck western Afghanistan, killing at least 100 people and wounding over 500 others. (October 3)
* Michael Kors presented his spring 2024 runway show in Brooklyn with famous faces like Halle Berry and T

In [12]:
query_engine.query("tell me more about the North Korea news").print_response_stream()

 Sure! Based on the provided context information, there are several pieces of news related to North Korea that I can provide more information on:

1. Kim Jong Un's trip to Russia: According to the Associated Press video, North Korea's leader Kim Jong Un rolled into Russia on an armored train Tuesday to meet with President Vladimir Putin. This is a rare meeting between the two leaders, who are both facing escalating standoffs with the West.
2. Flooding in eastern Libya: The AP video reports that thousands are feared dead and thousands more are missing

In [13]:
query_engine.query("any news about Trump?").print_response_stream()

 Based on the provided context information, there is no direct mention of Donald Trump in the given text. However, there are some mentions of the White House and Congress, which could potentially be related to Trump's presidency. Here are the relevant sentences:

* "The Associated Press explains what an impeachment inquiry entails now that House Speaker Kevin McCarthy wants to investigate President Joe Biden and his son Hunter as the election-year clash between Congress and the White House unfolds."
* "Heavy rainfall has flooded parts of Massachusetts and Rhode Island as water pou